In [1]:
import os

from rich import print as rprint

from entitysdk.client import Client
from entitysdk.common import ProjectContext
from entitysdk.models.morphology import (
    BrainLocation,
    BrainRegion,
    ReconstructionMorphology,
    Species,
    Strain,
)

### Initialize the entitycore client

In [2]:
entitycore_api_url = "http://127.0.0.1:8000"
project_context = ProjectContext(
    virtual_lab_id="103d7868-147e-4f07-af0d-71d8568f575c",
    project_id="103d7868-147e-4f07-af0d-71d8568f575c",
)
client = Client(api_url=entitycore_api_url, project_context=project_context)
token = os.getenv("ACCESS_TOKEN", "XXX")

## Search for entities

In [4]:
species = client.search(entity_type=Species, query={"name": "Mus musculus"}, token=token)[0]
rprint(species)

strain = client.search(entity_type=Strain, query={"name": "Cux2-CreERT2"}, token=token)[0]
rprint(strain)

dict_keys(['data', 'pagination', 'facets'])


Species(
    id=1,
    update_date=datetime.datetime(2025, 3, 1, 14, 12, 53, 73814, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 3, 1, 14, 12, 53, 73814, tzinfo=TzInfo(UTC)),
    authorized_public=None,
    authorized_project_id=None,
    name='Mus musculus',
    taxonomy_id='NCBITaxon:10090'
)

dict_keys(['data', 'pagination', 'facets'])


Strain(
    id=1,
    update_date=datetime.datetime(2025, 3, 1, 14, 13, 13, 164091, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 3, 1, 14, 13, 13, 164091, tzinfo=TzInfo(UTC)),
    authorized_public=None,
    authorized_project_id=None,
    name='Cux2-CreERT2',
    taxonomy_id='http://bbp.epfl.ch/neurosciencegraph/ontologies/speciestaxonomy/RBS4I6tyfUBSDt1i0jXLpgN',
    species_id=1
)

### Create a morphology object

In [6]:
# No GET endpoint yet for species

brain_location = BrainLocation(
    x=4101.52490234375,
    y=1173.8499755859375,
    z=4744.60009765625,
)
brain_region = BrainRegion(
    id=68,
    name="Frontal pole, layer 1",
    acronym="FRP1",
    children=[],
)

morphology = ReconstructionMorphology(
    name="my-morph",
    description="A morphology",
    species=species,
    strain=strain,
    brain_region=brain_region,
    location=brain_location,
    legacy_id="temp",
)

## Register it

In [7]:
registered = client.register(
    entity=morphology,
    token=token,
)
rprint(registered)

ReconstructionMorphology(
    id=8696,
    update_date=datetime.datetime(2025, 3, 1, 14, 50, 1, 518799, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 3, 1, 14, 50, 1, 518799, tzinfo=TzInfo(UTC)),
    authorized_public=False,
    authorized_project_id='103d7868-147e-4f07-af0d-71d8568f575c',
    name='my-morph',
    description='A morphology',
    pref_label=None,
    species=Species(
        id=1,
        update_date=datetime.datetime(2025, 3, 1, 14, 12, 53, 73814, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 3, 1, 14, 12, 53, 73814, tzinfo=TzInfo(UTC)),
        authorized_public=None,
        authorized_project_id=None,
        name='Mus musculus',
        taxonomy_id='NCBITaxon:10090'
    ),
    strain=Strain(
        id=1,
        update_date=datetime.datetime(2025, 3, 1, 14, 13, 13, 164091, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 3, 1, 14, 13, 13, 164091, tzinfo=TzInfo(UTC)),
        authorized_public=None,
        authorized_project_id=None,
        name='Cux2-CreERT2',
        taxonomy_id='http://bbp.epfl.ch/neurosciencegraph/ontologies/speciestaxonomy/RBS4I6tyfUBSDt1i0jXLpgN',
        species_id=1
    ),
    brain_region=BrainRegion(
        id=68,
        update_date=datetime.datetime(2025, 3, 1, 14, 12, 29, 635837, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 3, 1, 14, 12, 29, 635837, tzinfo=TzInfo(UTC)),
        authorized_public=None,
        authorized_project_id=None,
        name='Frontal pole, layer 1',
        acronym='FRP1',
        children=[]
    ),
    location=BrainLocation(x=4101.52490234375, y=1173.8499755859375, z=4744.60009765625),
    contributions=[],
    license=None,
    legacy_id=None
)

## Retrieve it

In [8]:
fetched = client.get(entity_id=registered.id, entity_type=ReconstructionMorphology, token=token)
rprint(fetched)

ReconstructionMorphology(
    id=8696,
    update_date=datetime.datetime(2025, 3, 1, 14, 50, 1, 518799, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 3, 1, 14, 50, 1, 518799, tzinfo=TzInfo(UTC)),
    authorized_public=False,
    authorized_project_id='103d7868-147e-4f07-af0d-71d8568f575c',
    name='my-morph',
    description='A morphology',
    pref_label=None,
    species=Species(
        id=1,
        update_date=datetime.datetime(2025, 3, 1, 14, 12, 53, 73814, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 3, 1, 14, 12, 53, 73814, tzinfo=TzInfo(UTC)),
        authorized_public=None,
        authorized_project_id=None,
        name='Mus musculus',
        taxonomy_id='NCBITaxon:10090'
    ),
    strain=Strain(
        id=1,
        update_date=datetime.datetime(2025, 3, 1, 14, 13, 13, 164091, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 3, 1, 14, 13, 13, 164091, tzinfo=TzInfo(UTC)),
        authorized_public=None,
        authorized_project_id=None,
        name='Cux2-CreERT2',
        taxonomy_id='http://bbp.epfl.ch/neurosciencegraph/ontologies/speciestaxonomy/RBS4I6tyfUBSDt1i0jXLpgN',
        species_id=1
    ),
    brain_region=BrainRegion(
        id=68,
        update_date=datetime.datetime(2025, 3, 1, 14, 12, 29, 635837, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 3, 1, 14, 12, 29, 635837, tzinfo=TzInfo(UTC)),
        authorized_public=None,
        authorized_project_id=None,
        name='Frontal pole, layer 1',
        acronym='FRP1',
        children=[]
    ),
    location=BrainLocation(x=4101.52490234375, y=1173.8499755859375, z=4744.60009765625),
    contributions=[],
    license=None,
    legacy_id=None
)

## Search it

In [9]:
hits = client.search(
    entity_type=ReconstructionMorphology,
    query={"name": "my-morph", "page": 1, "page_size": 2},
    token=token,
)
for hit in hits:
    rprint(hit)

dict_keys(['data', 'pagination', 'facets'])


ReconstructionMorphology(
    id=8696,
    update_date=datetime.datetime(2025, 3, 1, 14, 50, 1, 518799, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 3, 1, 14, 50, 1, 518799, tzinfo=TzInfo(UTC)),
    authorized_public=False,
    authorized_project_id='103d7868-147e-4f07-af0d-71d8568f575c',
    name='my-morph',
    description='A morphology',
    pref_label=None,
    species=Species(
        id=1,
        update_date=datetime.datetime(2025, 3, 1, 14, 12, 53, 73814, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 3, 1, 14, 12, 53, 73814, tzinfo=TzInfo(UTC)),
        authorized_public=None,
        authorized_project_id=None,
        name='Mus musculus',
        taxonomy_id='NCBITaxon:10090'
    ),
    strain=Strain(
        id=1,
        update_date=datetime.datetime(2025, 3, 1, 14, 13, 13, 164091, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 3, 1, 14, 13, 13, 164091, tzinfo=TzInfo(UTC)),
        authorized_public=None,
        authorized_project_id=None,
        name='Cux2-CreERT2',
        taxonomy_id='http://bbp.epfl.ch/neurosciencegraph/ontologies/speciestaxonomy/RBS4I6tyfUBSDt1i0jXLpgN',
        species_id=1
    ),
    brain_region=BrainRegion(
        id=68,
        update_date=datetime.datetime(2025, 3, 1, 14, 12, 29, 635837, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 3, 1, 14, 12, 29, 635837, tzinfo=TzInfo(UTC)),
        authorized_public=None,
        authorized_project_id=None,
        name='Frontal pole, layer 1',
        acronym='FRP1',
        children=[]
    ),
    location=BrainLocation(x=4101.52490234375, y=1173.8499755859375, z=4744.60009765625),
    contributions=[],
    license=None,
    legacy_id=None
)

ReconstructionMorphology(
    id=2104,
    update_date=datetime.datetime(2024, 12, 4, 13, 54, 29, 266016, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2024, 11, 25, 10, 49, 10, 183426, tzinfo=TzInfo(UTC)),
    authorized_public=False,
    authorized_project_id='103d7868-147e-4f07-af0d-71d8568f575c',
    name='18864_00528',
    description='Initial neuron morphology shared by provider',
    pref_label=None,
    species=Species(
        id=1,
        update_date=datetime.datetime(2025, 3, 1, 14, 12, 53, 73814, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 3, 1, 14, 12, 53, 73814, tzinfo=TzInfo(UTC)),
        authorized_public=None,
        authorized_project_id=None,
        name='Mus musculus',
        taxonomy_id='NCBITaxon:10090'
    ),
    strain=Strain(
        id=1,
        update_date=datetime.datetime(2025, 3, 1, 14, 13, 13, 164091, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 3, 1, 14, 13, 13, 164091, tzinfo=TzInfo(UTC)),
        authorized_public=None,
        authorized_project_id=None,
        name='Cux2-CreERT2',
        taxonomy_id='http://bbp.epfl.ch/neurosciencegraph/ontologies/speciestaxonomy/RBS4I6tyfUBSDt1i0jXLpgN',
        species_id=1
    ),
    brain_region=BrainRegion(
        id=2511156654,
        update_date=datetime.datetime(2025, 3, 1, 14, 12, 29, 701483, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 3, 1, 14, 12, 29, 701483, tzinfo=TzInfo(UTC)),
        authorized_public=None,
        authorized_project_id=None,
        name='Secondary motor area, layer 3',
        acronym='MOs3',
        children=[]
    ),
    location=BrainLocation(x=4185.5498046875, y=1452.9000244140625, z=7027.07568359375),
    contributions=[
        Contribution(
            agent=Organization(
                id=7,
                update_date=datetime.datetime(2022, 4, 11, 8, 28, 4, 176000, tzinfo=TzInfo(UTC)),
                creation_date=datetime.datetime(2020, 8, 28, 11, 43, 59, 117000, tzinfo=TzInfo(UTC)),
                type='organization',
                pref_label='Southeast University',
                alternative_name=''
            ),
            role=Role(
                id=1,
                update_date=datetime.datetime(2025, 3, 1, 14, 12, 53, 97423, tzinfo=TzInfo(UTC)),
                creation_date=datetime.datetime(2025, 3, 1, 14, 12, 53, 97423, tzinfo=TzInfo(UTC)),
                name='unspecified',
                role_id='unspecified'
            )
        )
    ],
    license=None,
    legacy_id=None
)